In [ ]:
from langchain.agents import ZeroShotAgent, Tool, AgentExecutor, create_sql_agent, get_all_tool_names
from langchain.chains import SQLDatabaseChain
from langchain import OpenAI, SerpAPIWrapper, LLMChain, GoogleSearchAPIWrapper
from langchain.agents.agent_toolkits import SQLDatabaseToolkit
from langchain.sql_database import SQLDatabase

In [ ]:
# llm = OpenAI(
#     temperature=0,
    
# )

# tools = load_tools(["google-search", "llm-math"], llm=llm)

# agent = initialize_agent(tools, llm, agent="zero-shot-react-description", verbose=True)


In [ ]:
get_all_tool_names()

In [ ]:
google_search = GoogleSearchAPIWrapper(k=10, search_engine="google")

In [ ]:
google_search = GoogleSearchAPIWrapper(k=10, search_engine="google")
sql_dk_co2_db = SQLDatabase.from_uri("sqlite:///data/dk_co2_emission.db")
# toolkit = SQLDatabaseToolkit(db=sql_dk_co2_db)

llm=OpenAI(temperature=0)
db_chain = SQLDatabaseChain(llm=llm, database=sql_dk_co2_db, verbose=True)

db_chain.run(
    """
    What is the total emission of a red lentils in kg CO2e per kg?
    Use the following query format: SELECT "Total-kg-CO2-eq-kg" FROM dk_co2_emission WHERE "Name" LIKE "{item}"
    """,
)

In [ ]:
from langchain.agents import ZeroShotAgent, Tool, AgentExecutor
from langchain import OpenAI, SerpAPIWrapper, LLMChain
from langchain.utilities import PythonREPL

In [ ]:
sql_dk_co2_db = SQLDatabase.from_uri("sqlite:///data/dk_co2_emission.db")
sqlite_search = SQLDatabaseChain(llm=llm, database=sql_dk_co2_db, verbose=True)
google_search = GoogleSearchAPIWrapper(k=10, search_engine="google")
python_repl = PythonREPL()


tools = [
    Tool(
        name="CO2 emission database search",
        func=sqlite_search.run,
        description=f"Searches the CO2 emission database from an ingredient. The database is based on the Danish food database. Use the following query format: SELECT 'Total-kg-CO2-eq-kg' FROM dk_co2_emission WHERE 'Name' LIKE 'ingredient'",
    ),
    Tool(
        name="Google search",
        func=google_search.run,
        description="""Useful for finding out the CO2 emission of items or getting the weights from 
                       food ingredients such as one tablespoon is equal to 21 grams""",
    ),
    Tool(
        name="Python",
        func=python_repl.run,
        description="""Useful for when calculating the total CO2 emission of all the ingredients in a recipe and calculating the weights of each item."""
    )
]

In [ ]:
prefix = """Act as a CO2 emission calculator that understands danish and english. Given a list of ingredients to a recipe, find the total CO2 emission of the recipe in kg CO2e per kg. 
Extract the ingredients one by one and find the CO2 emission of each ingredient in kg. Then sum up the total CO2 emission of the recipe in kg CO2e per kg. 

First search the items in the ingredients list in the database and then use Google to find the weights of the ingredients given the units in the recipe. 
If the items are danish, then translate them to english and search the database and google using the english name.

Example:
We are given two items in the ingredients list: "150 røde linser, 2 gulerødder".

To search we translate the danish items to english and use the following query: SELECT "Total-kg-CO2-eq-kg" FROM dk_co2_emission WHERE "Name" LIKE '%red lentils%' AND Name LIKE '%carrot%'
We get 1.78 kg CO2e per kg for red lentils and 0.255 kg CO2e per kg for carrots. Next we use Google to find the weights of the carrot. 
1 carrots approximately weighs 0,060 kg or 60 grams according to a google search and 2 carrots weighs 0.120 kg.

Therefore, the total emission is 0.150 kg * 1.78 kg CO2e per kg + 0.120 kg * 0.255 kg CO2e per kg = 0.360 kg CO2e per kg.

This is the total CO2 emission of the recipe in kg CO2e per kg. This example only has two items in the ingredients list, but you should be able to handle any number of items in the ingredients list.
You should search for multiple items in the database and google using the same query.
 
You have access to the following tools:"""

suffix = """First search the items in the CO2 database and then use Google Search to find the weights of the ingredients given the units in the recipe. 

If you cannot find the item in the database then use Google Search to find the CO2 emission of the item.
Remember to use the units in the recipe to find the weights of the ingredients. 

Provide the answer in kg CO2e per kg and to calculate the total CO2 emission of all the ingredients in the recipe.
Remember to search the database using the command: SELECT "Total-kg-CO2-eq-kg" FROM dk_co2_emission WHERE "Name" LIKE "ingredient"
and translate from danish to english if the item is danish.

Begin! Calculate kg CO2e per kg for all items in the recipe and sum them: {input}

Question: {input}
{agent_scratchpad}"""

prompt = ZeroShotAgent.create_prompt(
    tools, 
    prefix=prefix, 
    suffix=suffix, 
    input_variables=["input", "agent_scratchpad"]
)

In [ ]:
llm_chain = LLMChain(llm=OpenAI(temperature=0), prompt=prompt)

In [ ]:
tool_names = [tool.name for tool in tools]
agent = ZeroShotAgent(llm_chain=llm_chain, allowed_tools=tool_names)

agent_executor = AgentExecutor.from_agent_and_tools(agent=agent, tools=tools, verbose=True)

In [ ]:
agent_executor.run(
    """150 g røde linser
    1 stort løg, 1 peberfrugt
    2 terninger grøntsagsbouillon
    3 fed hvidløg, pressede
    1 dåse hakkede tomater, 
    1 dåse tomatkoncentrat (140 g), 
    1 dåse kokosmælk (400 ml), 
    1 spsk. citronsaft, 
    1 tsk. sukker, 
    ½ tsk. chilipulver, 
    ½ tsk. stødt spidskommen, 
    Evt. 1/4 tsk. røget paprika, 
    En stor håndfuld frisk basilikum, hakket
    (eller 2 tsk. tørret basilikum)"""
)